# Loading Data
In lessons `02` and `03` we explored what data makes up an Evaluation dataset and how it can be accessed using the Evaluation table sub-classes.  In lesson `02` we cloned a complete dataset from AWS S3 to explore. In this lesson we are going to explore how your data can be added to a new or existing Evaluation dataset.

Similar to previous lessons we will start by creating a Evaluation class instance that references a directory where the data will be stored, and then clone the Evalation template as a starting point.

In [ ]:
import teehr
from pathlib import Path
import shutil

# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "04_loading_data")
shutil.rmtree(test_eval_dir, ignore_errors=True)

# Create an Evaluation object and create the directory
ev = teehr.Evaluation(dir_path=test_eval_dir, create_dir=True)

# Clone the template
ev.clone_template()

For this example, we will use the data from the example dataset. The data is stored in the `tests/data` folder of the teehr repository. We will copy the data to the Evaluation directory, examine the contents and then load into the Evaluation dataset.  Note these data are for demonstration purposes only.  We have given the data fake location IDs and fake timeserries values for demonstration purposes. We will work with real data in future exercises.

First lets download some location data.  We will download the `gages.geojson` file which is a GeoJSON file that contains 3 fictional gages.

In [ ]:
!wget https://github.com/RTIInternational/teehr/raw/refs/heads/main/tests/data/v0_3_test_study/geo/gages.geojson \
    -O $HOME/temp/04_loading_data/gages.geojson

Now lets open the `gages.geojson` file with GeoPandas and take a look at the contents.

In [ ]:
import geopandas as gpd
location_data_path = Path(test_eval_dir, "gages.geojson")
gdf = gpd.read_file(location_data_path)
gdf

You can see that the file contains 3 gages that are each represnted by a point.  Next we will load the location data into the TEEHR dataset using methods on the `locations` table sub-class. Note that this file has the `id`, `name` and `geometry` fields required by TEEHR.  If it didn't, the `load_spatial` method can take a `field_mapping` dictionary that can be used to specify which fields should be mapped to which required field name.

In [ ]:
ev.locations.load_spatial(location_data_path)

Now that the location data has been loaded into the TEEHR dataset, it can be queried with the `ev.locations` class methods, but more importantly, now we can load timeseries that references the `id` in the locations data.  The `id` in the `locations` table is the primary location ID and is refrenced in the `primary_timeseries` table as `location_id`.

First lets query the location data to verify it has been loaded, then look at a map to verify it is in the correct locations, then we will move on loading some timeseries data.

In [ ]:
locations_gdf = ev.locations.to_geopandas()
locations_gdf

This table of locations came from the TEEHR dataset and matches what we saw in the source file, so we know it was loaded successfuly.  Now lets look at a plot using the built-in teehr.location_map() method.

In [ ]:
# Tell Bokeh to output plots in the notebook
from bokeh.io import output_notebook
output_notebook()

locations_gdf.teehr.location_map()

Ok, now that we have locaded the location data, we can load some timeseries to the `primary_timeseries` table.  The primary timeseries represents the timeseries that simulations will be compared to (i.e., the truth).

Let's download a test timeseries from the TEEHR respository as a *.CSV file, open and examine the contents with Pandas, and load into the TEEHR dataset.

In [ ]:
!wget https://github.com/RTIInternational/teehr/raw/refs/heads/main/tests/data/v0_3_test_study/timeseries/test_short_obs.csv \
    -O $HOME/temp/04_loading_data/test_short_obs.csv

In [ ]:
import pandas as pd
primary_timeseries_path = Path(test_eval_dir, "test_short_obs.csv")
primary_timeseries_df = pd.read_csv(primary_timeseries_path)
primary_timeseries_df.head()

There a few things we need to look at and consider before loading the data into the TEEHR dataset.  First is the field names.  You can see above that while the schema of this dataset is close to the TEEHR schema, it is not exactly correct, specifically `configuration` should be `configuration_name` and `measurement_unit` should be `unit_name`.  To address this situation we can use the `field_mapping` argument to mapp the existing field names to the required field names.  The next thing that needs to be delt with are the field values for fields that reference values in another table, for example a domain table or the location table.  Lets start with `unit_name`.  The `unit_name` values need to reference a value from the `name` field in the `units` table.  Next lets look at what values are in the `uints` table.

In [ ]:
ev.units.to_pandas()

Here we can see that `m^3/s` is in the `units` table, so we are all set with values in the `measurement_units` field.  Next lets check the `variables` table.

In [ ]:
ev.variables.to_pandas()

Here we see that `streamflow` is not in the `name` field of the `variables` table.  There are two ways to remedy this.  1) we could add the value `streamflow` to the `variables` table or 2) we can use the `constant_field_values` argument to set the `variable_name` to an allowed values. In this case we will go with the latter and set the `variable_name` to `streamflow_hourly_inst`.  Next we need to address the `configuration_name` field.  First lets see what values are in the `configurations` table.  Because the `configurations` table is rather large, we will search it for `test_obs` to see if that value is in there (it isn't).

In [ ]:
ev.configurations.filter(
    {
        "column": "name",
        "operator": "=",
        "value": "test_obs"
    }
).to_pandas()

We can see by the empty DataFranme that a configuration named `test_obs` is not in the table.  Again, we have two option in this case, we could use the `constant_field_values` argument to set the `configuration_name` to a value that is in the table (like `usgs_observations`) or we could add a new row to the `configurations` table.  Again we will use the `constant_field_values` argument to set the value to a constant.  When we insert the `secondary_timeseries` below we will add a new row to the `configurations` table.

In [ ]:
# Load the timeseries data and map over the fields and set constants
ev.primary_timeseries.load_csv(
    in_path=primary_timeseries_path,
    field_mapping={
        "reference_time": "reference_time",
        "value_time": "value_time",
        "configuration": "configuration_name",
        "measurement_unit": "unit_name",
        "variable_name": "variable_name",
        "value": "value",
        "location_id": "location_id"
    },
    constant_field_values={
        "unit_name": "m^3/s",
        "variable_name": "streamflow_hourly_inst",
        "configuration_name": "usgs_observations"
    }
)

Lets do a query for a single timeseries and make sure the data was loaded.

In [ ]:
primary_timeseries_df = ev.primary_timeseries.filter(
    {
        "column": "location_id",
        "operator": "=",
        "value": "gage-A"
    }
).to_pandas()
primary_timeseries_df.head()

In [ ]:
primary_timeseries_df.teehr.timeseries_plot()

Next lets load some simulated data that we want to compare to the observed timeseries (primary timeseries).  The secondary timeseries table is where the "simulated" timeseries data are stored.  First we will download some test secondary timeseries from the TEEHR repository.

In [ ]:

!wget https://github.com/RTIInternational/teehr/raw/refs/heads/main/tests/data/v0_3_test_study/timeseries/test_short_fcast.parquet \
    -O $HOME/temp/04_loading_data/test_short_fcast.parquet


In [ ]:
secondary_timeseries_path = Path(test_eval_dir, "test_short_fcast.parquet")
secondary_timeseries_df = pd.read_parquet(secondary_timeseries_path)
secondary_timeseries_df.head()

When loading `secondary_timeseries`, the same considerations regarding field names and field values apply as those for `primary_timeseries`.  Unlike when we loaded the `primary_timeseries`, this time we will add a new configuration to the `configurations` table.  This is more common that the `configuration_name` for the secondary timeseries that you are loading is not in the `configurations` table.

In [ ]:
ev.configurations.add(
    teehr.Configuration(
        name="test_short",
        type="secondary",
        description="Test Forecast Configuration",
    )
)

There is one other consideration before we can load the `secondary_timeseries`.  There is no expectation in the TEEHR schema that the `location_id` used of the `primary_timeseries` and that used for `secondary_timeseries` match.  Therefore, we need to load a location crosswalk data to the `location_crosswalks` table so that TEEHR knows which `primary_location_id` matches which `secondary_location_id`. For example, in our fake test dataset, the observation data at `location_id` "gage-A" relates to forecasts with a `location_id` "fcst-1".  

We already have a file with this information, so like we did with the other test datasets, we will download the data from the TEEHR repository, open it with Pandas to checkout the contents and load into the TEEHR dataset.

In [ ]:
!wget https://github.com/RTIInternational/teehr/raw/refs/heads/main/tests/data/v0_3_test_study/geo/crosswalk.csv \
    -O $HOME/temp/04_loading_data/crosswalk.csv

In [ ]:
crosswalk_path = Path(test_eval_dir, "crosswalk.csv")
crosswalk_df = pd.read_csv(crosswalk_path)
crosswalk_df

In this case we can see that the crosswalk table already has the field names we need (`primary_location_id` and `secondary_location_id`) so we don't need to use the `field_mapping` argument to pass the loading function a field mapping.

In [ ]:
# Load the crosswalk data to the crosswalks table
ev.location_crosswalks.load_csv(crosswalk_path)

In [ ]:
# Load the secondary timeseries data and map over the fields and set constants
ev.secondary_timeseries.load_parquet(
    in_path=secondary_timeseries_path,
    field_mapping={
        "reference_time": "reference_time",
        "value_time": "value_time",
        "configuration": "configuration_name",
        "measurement_unit": "unit_name",
        "variable_name": "variable_name",
        "value": "value",
        "location_id": "location_id"
    },
    constant_field_values={
        "unit_name": "m^3/s",
        "variable_name": "streamflow_hourly_inst"
    }
)

In [ ]:
# Query the secondary timeseries data for a single forecast.
secondary_timeseries_df = (
    ev.secondary_timeseries
    .filter(
        {
            "column": "reference_time",
            "operator": "=",
            "value": "2022-01-01"
        }
    )
).to_pandas()
secondary_timeseries_df.head()

In [ ]:
secondary_timeseries_df.teehr.timeseries_plot()

At this point we have added data to the following TEEHR data tables:
- locations
- configurations
- location_crosswalks 
- primary_timeseries
- secondary_timeseries

This is the bare minimum needed to conduct an evaluation, but we are going to add one more type of data, `location_attributes`.  Location attributes are that provide some type of extra information for the primary locations.  These are data are anologous to the columns in a geospatial data file "attribute table" but are stored in a seperate table.

In [ ]:
!wget https://github.com/RTIInternational/teehr/raw/refs/heads/main/tests/data/v0_3_test_study/geo/test_attr_2yr_discharge.csv \
    -O $HOME/temp/04_loading_data/test_attr_2yr_discharge.csv

!wget https://github.com/RTIInternational/teehr/raw/refs/heads/main/tests/data/v0_3_test_study/geo/test_attr_drainage_area_km2.csv \
    -O $HOME/temp/04_loading_data/test_attr_drainage_area_km2.csv

!wget https://github.com/RTIInternational/teehr/raw/refs/heads/main/tests/data/v0_3_test_study/geo/test_attr_ecoregion.csv \
    -O $HOME/temp/04_loading_data/test_attr_ecoregion.csv

Like before, lets take a look at the raw data we downloaded from the repository.

In [ ]:
# Read and display the 3 attribute table contents
attr_2yr_discharge_df = pd.read_csv(Path(test_eval_dir, "test_attr_2yr_discharge.csv"))
display(attr_2yr_discharge_df)

attr_drainage_area_km2_df = pd.read_csv(Path(test_eval_dir, "test_attr_drainage_area_km2.csv"))
display(attr_drainage_area_km2_df)

attr_ecoregion_df = pd.read_csv(Path(test_eval_dir, "test_attr_ecoregion.csv"))
display(attr_ecoregion_df)


We can see we have data for three different attributes here, `year_2_discharge`,`drainage_area`, `ecoregion`.  Before we can load this data we need to add the attribute names to the `attributes` table and then we can add the location attribute values to the `location_attributes` table.  Like we did for adding configurations, we can use the `ev.attributes.add()` method to add new values.

In [ ]:
# Add some attributes
ev.attributes.add(
    [
        teehr.Attribute(
            name="drainage_area",
            type="continuous",
            description="Drainage area in square kilometers"
        ),
        teehr.Attribute(
            name="ecoregion",
            type="categorical",
            description="Ecoregion"
        ),
        teehr.Attribute(
            name="year_2_discharge",
            type="continuous",
            description="2-yr discharge in cubic meters per second"
        ),
    ]
)

In [ ]:
# Load the location attribute data
ev.location_attributes.load_csv(
    in_path=test_eval_dir,
    field_mapping={"attribute_value": "value"},
    pattern="test_attr_*.csv",
)

In [ ]:
# Query the location attribute data
ev.location_attributes.to_pandas()

Now we have data in all the tables required to conduct and evaluation.  There is one more step to do before we can start "slicing and dicing" the data and looking at evaluation metrics - we need to generate the `joined_timeseries` data table.  The `joined_timeseries` table is essentially a materialized view that joins the primary and secondary timeseries joined together based on `location_id`, `value_time`, `variable_name` and `unit_name`, adds the `location_attributes` and adds any user defined fields.

In [ ]:
!cat $HOME/temp/04_loading_data/scripts/user_defined_fields.py

In [ ]:
ev.joined_timeseries.create(execute_udf=True)

In [ ]:
ev.joined_timeseries.to_pandas().head()

In [ ]:
ev.joined_timeseries.to_sdf().columns

This concludes this lesson.  The next lesson will 